# Hands-on Lab: Complete the EDA with SQL

## Assignment: SQL Notebook for Peer Assignment

### Import the required modules

In [1]:
import pandas as pd
import csv, sqlite3
import sqlalchemy
import numpy as np

### Connect to the database

In [2]:
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [3]:
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

### Task 1: Display the names of the unique launch sites in the space mission

In [4]:
cur.execute('select DISTINCT Launch_Site from SPACEXTBL')
results = cur.fetchall()
for row in results:
    print(row)

('CCAFS LC-40',)
('VAFB SLC-4E',)
('KSC LC-39A',)
('CCAFS SLC-40',)


### Task 2: Display 5 records where launch sites begin with the string 'CCA'

In [5]:
cur.execute('select * from SPACEXTBL WHERE Launch_Site LIKE "CCA%" LIMIT 5')
results = cur.fetchall()
for row in results:
    print(row)

('2010-06-04', '18:45:00', 'F9 v1.0  B0003', 'CCAFS LC-40', 'Dragon Spacecraft Qualification Unit', 0, 'LEO', 'SpaceX', 'Success', 'Failure (parachute)')
('2010-12-08', '15:43:00', 'F9 v1.0  B0004', 'CCAFS LC-40', 'Dragon demo flight C1, two CubeSats, barrel of Brouere cheese', 0, 'LEO (ISS)', 'NASA (COTS) NRO', 'Success', 'Failure (parachute)')
('2012-05-22', '7:44:00', 'F9 v1.0  B0005', 'CCAFS LC-40', 'Dragon demo flight C2', 525, 'LEO (ISS)', 'NASA (COTS)', 'Success', 'No attempt')
('2012-10-08', '0:35:00', 'F9 v1.0  B0006', 'CCAFS LC-40', 'SpaceX CRS-1', 500, 'LEO (ISS)', 'NASA (CRS)', 'Success', 'No attempt')
('2013-03-01', '15:10:00', 'F9 v1.0  B0007', 'CCAFS LC-40', 'SpaceX CRS-2', 677, 'LEO (ISS)', 'NASA (CRS)', 'Success', 'No attempt')


### Task 3: Display the total payload mass carried by boosters launched by NASA (CRS)

In [6]:
cur.execute("select SUM(PAYLOAD_MASS__KG_) from SPACEXTBL WHERE Customer='NASA (CRS)'")
results = cur.fetchall()
results

[(45596,)]

### Task 4: Display average payload mass carried by booster version F9 v1

In [7]:
cur.execute("select AVG(PAYLOAD_MASS__KG_) from SPACEXTBL WHERE Booster_Version='F9 v1.1'")
results = cur.fetchall()
results

[(2928.4,)]

### Task 5: List the date when the first succesful landing outcome in ground pad was achieved.

In [8]:
cur.execute("select MIN(Date) from SPACEXTBL WHERE Mission_Outcome='Success'")
results = cur.fetchall()
results

[('2010-06-04',)]

### Task 6: List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

In [9]:
cur.execute("select Booster_Version from SPACEXTBL WHERE Landing_Outcome='Success (drone ship)' AND (PAYLOAD_MASS__KG_ BETWEEN 4000 AND 6000)")
results = cur.fetchall()
for result in results:
    print(result)

('F9 FT B1022',)
('F9 FT B1026',)
('F9 FT  B1021.2',)
('F9 FT  B1031.2',)


### Task 7: List the total number of successful and failure mission outcomes

In [10]:
cur.execute('select Mission_Outcome, COUNT(*) from SPACEXTBL GROUP BY Mission_Outcome')
results = cur.fetchall()
for row in results:
    print(row)

('Failure (in flight)', 1)
('Success', 98)
('Success ', 1)
('Success (payload status unclear)', 1)


### Task 8: List the names of the booster_versions which have carried the maximum payload mass. Use a subquery

In [11]:
cur.execute('select Booster_Version from SPACEXTBL WHERE PAYLOAD_MASS__KG_ = (select MAX(PAYLOAD_MASS__KG_) FROM SPACEXTBL)')
results = cur.fetchall()
for row in results:
    print(row)

('F9 B5 B1048.4',)
('F9 B5 B1049.4',)
('F9 B5 B1051.3',)
('F9 B5 B1056.4',)
('F9 B5 B1048.5',)
('F9 B5 B1051.4',)
('F9 B5 B1049.5',)
('F9 B5 B1060.2 ',)
('F9 B5 B1058.3 ',)
('F9 B5 B1051.6',)
('F9 B5 B1060.3',)
('F9 B5 B1049.7 ',)


### Task 9: List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

In [15]:
cur.execute("SELECT substr(Date, 6, 2) AS Month, Landing_Outcome, Booster_Version, Launch_Site FROM SPACEXTBL WHERE \
             substr(Date, 1, 4) = '2015' AND Landing_Outcome LIKE '%Failure (drone ship)%'")
results = cur.fetchall()
for row in results:
    print(row)

('01', 'Failure (drone ship)', 'F9 v1.1 B1012', 'CCAFS LC-40')
('04', 'Failure (drone ship)', 'F9 v1.1 B1015', 'CCAFS LC-40')


### Task 10: Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.¶

In [16]:
cur.execute("select Landing_Outcome, COUNT(*) from SPACEXTBL GROUP BY Landing_Outcome \
            HAVING (Date BETWEEN '2010-06-04' AND '2017-03-20') ORDER BY COUNT(*) DESC")
results = cur.fetchall()
for row in results:
    print(row)

('No attempt', 21)
('Success (drone ship)', 14)
('Success (ground pad)', 9)
('Failure (drone ship)', 5)
('Controlled (ocean)', 5)
('Uncontrolled (ocean)', 2)
('Failure (parachute)', 2)
('Precluded (drone ship)', 1)
